In [1]:
#Importing all dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
# New Dependency! Use this to pretty print the JSON
# https://docs.python.org/3/library/pprint.html
from pprint import pprint
# Note that the ?t= is a query param for the t-itle of the
# movie we want to search for.
url = "http://www.omdbapi.com/?t="
api_key = "&apikey=19ee31e3"

In [2]:
#Importing .csv file that contains movie names etc.
file_one = "../output/movie_counts.csv"
df = pd.read_csv(file_one, encoding="ISO-8859-1")
df.head()
movies = df["Movie_Title"].drop_duplicates().values.tolist()
len(movies)

1092

In [3]:
# Retrieving data about the movies from OMDB API and filtering all data out that has no IMDB info
data = []
modata = []
for k in movies:
    response = requests.get(url + k + api_key)
    data.append(response.json())
try:
    for l in range(0,len(data)):
        if data[l]['Response'] != 'False' :
            modata.append(data[l])
except KeyError:
    pass
I = [8,24,25,73,88,95,142,156,260,288,368,461,472,548,557,591,658,667,671,674,687,735,752,761,867,876,880,894,900,915]
modata = np.delete(modata, I).tolist()
len(modata)

C:\Users\Zach\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: in the future out of bounds indices will raise an error instead of being ignored by `numpy.delete`.
  del sys.path[0]


818

In [4]:
#Appending all IMDB ratings into IMDB array
IMDB = []
try:   
    for l in range(0,len(modata)):
                  IMDBm = round((float(modata[l]['Ratings'][0]['Value'].split('/')[0])/10),2)
                  IMDB.append(IMDBm)
except KeyError:
    pass
len(IMDB)

818

In [5]:
#Retrieving the movie titles,film ratings, released, runtimes, genre from the data object which is a list of ditionaries.
Movietitle = []
Filmrating = []
Released = []
Runtime = []
Genre = []
for i in range(len(modata)):
    Movietitle.append(modata[i]['Title'])
    Filmrating.append(modata[i]['Rated'])
    Released.append(modata[i]['Released'])
    Runtime.append(modata[i]['Runtime'])
    Genre.append(modata[i]['Genre'])
    
df2 = pd.DataFrame({'Movie_Title' : Movietitle,'IMDB':IMDB,'Film Rating':Filmrating,'Released':Released,'Runtime':Runtime,'Genre':Genre}, columns=['Movie_Title','IMDB','Film Rating','Released','Runtime','Genre'])
len(df2)

818

In [7]:
#The below dataframe has the data scraped from Worst Movie List
df.head()

,Movie_Title,Movie_Links,shit,fuck,damn,bitch,asshole,ass,dick,pussy,cock,bastard,douche,total_words
0,10 Things I Hate About You,/10-Things-I-Hate-About-You.html,8,2,3,5,3,7,2,1,0,0,0,19152
1,12,/12.html,1,0,1,0,1,0,0,1,0,2,0,14144
2,12 and Holding,/12-and-Holding.html,10,4,3,1,5,4,0,7,0,1,0,18999
3,12 Monkeys,/12-Monkeys.html,4,5,2,2,1,3,0,1,0,2,0,32272
4,12 Years a Slave,/12-Years-a-Slave.html,0,0,12,1,0,0,0,0,0,3,0,31014


In [8]:
#The below dataframe has the data scraped from OMDB API
df2.head()

,Movie_Title,IMDB,Film Rating,Released,Runtime,Genre
0,10 Things I Hate About You,0.73,PG-13,31 Mar 1999,97 min,"Comedy, Drama, Romance"
1,12,0.77,PG-13,20 Sep 2007,159 min,"Crime, Drama, Thriller, War"
2,12 and Holding,0.76,R,07 Jul 2006,94 min,Drama
3,12 Monkeys,0.77,TV-14,16 Jan 2015,42 min,"Adventure, Drama, Mystery, Sci-Fi, Thriller"
4,12 Years a Slave,0.81,R,08 Nov 2013,134 min,"Biography, Drama, History"


In [9]:
#We merge the df and df2 dataframes using left join on Movie Title column which is the common column between the 2 dataframes.
result = pd.merge(df2, df, how='left', on= 'Movie_Title')
result.head()

,Movie_Title,IMDB,Film Rating,Released,Runtime,Genre,Movie_Links,shit,fuck,damn,bitch,asshole,ass,dick,pussy,cock,bastard,douche,total_words
0,10 Things I Hate About You,0.73,PG-13,31 Mar 1999,97 min,"Comedy, Drama, Romance",/10-Things-I-Hate-About-You.html,8.0,2.0,3.0,5.0,3.0,7.0,2.0,1.0,0.0,0.0,0.0,19152.0
1,12,0.77,PG-13,20 Sep 2007,159 min,"Crime, Drama, Thriller, War",/12.html,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,14144.0
2,12 and Holding,0.76,R,07 Jul 2006,94 min,Drama,/12-and-Holding.html,10.0,4.0,3.0,1.0,5.0,4.0,0.0,7.0,0.0,1.0,0.0,18999.0
3,12 Monkeys,0.77,TV-14,16 Jan 2015,42 min,"Adventure, Drama, Mystery, Sci-Fi, Thriller",/12-Monkeys.html,4.0,5.0,2.0,2.0,1.0,3.0,0.0,1.0,0.0,2.0,0.0,32272.0
4,12 Years a Slave,0.81,R,08 Nov 2013,134 min,"Biography, Drama, History",/12-Years-a-Slave.html,0.0,0.0,12.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,31014.0


In [10]:
#We export the merged dataframe into a result.csv file
result.to_csv('../output/result.csv')

In [1]:
#We retrieve the result.csv file and create 2 columns - Runtime minutes and total_bad_words
file_two = "../output/result.csv"
df3 = pd.read_csv(file_two, encoding="ISO-8859-1")
df3.head()
df3.fillna('float(0)', inplace = True)
df3['total_bad_words'] = df3['shit'] + df3['fuck'] + df3['damn'] + df3['bitch']+df3['asshole']+df3['ass']+df3['dick']+df3['pussy']+df3['cock']+df3['bastard']+df3['douche']
df3['total_bad_words']
Run = df3["Runtime"].tolist()
for i in range(0,len(Run)):
    Run[i] = Run[i].split()[0]
#We split the Runtime column to get Runtime_minutes column
df3["Runtime_minutes"] = [float(1) if str(x)=='float(0)' else float(x) for x in Run]
bad = df3["total_bad_words"].tolist()
#Cleaning the dataframe
for i in range(len(bad)):
    if bad[i] == 'float(0)float(0)float(0)float(0)float(0)float(0)float(0)float(0)float(0)float(0)float(0)':
        bad[i] = 1 
    bad[i] = float(bad[i])
df3["total_bad_words"] = bad
#After converting the columntype for columns - Runtime_minutes and total_bad_words, we create a new column - bad_word_frequency.
df3["bad_word_frequency"] = round(df3["Runtime_minutes"]/df3["total_bad_words"],2)
df3.head()
#Dropping all null columns
df3 = df3.drop(df3.index[[13,75,224,592,780,784,800]])
df3.reset_index()
df3 = df3[df3.Released != 'float(0)']
#Creating a year column after splitting the Released column
R = df3['Released'].tolist()
for i in range (len(R)):
    R[i] = int(R[i].split()[2])
df3["Year"] = R
#Removing all inf and NaN rows, thus cleaning the dataframe before generating visualizations
df3 = df3.replace(np.inf, 0.00000001)
df3 = df3[df3.bad_word_frequency != 0.00000001]
df3 = df3[df3.douche != 'float(0)']
df3.reset_index()
#Outputing the cleaned dataframe df3 to results.csv file
df3.to_csv('../output/result2.csv')

NameError: name 'pd' is not defined